# Search problem: Beijing Subway

## 1.Python Crawler: Beijing Subway

In [1]:
import requests, re, urllib.request

In [2]:
def get_reponse_TEXT(url):
    """
    获取url连接中的页面信息TEXT
    """

    headers = {"User-Agent": "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}
    reponse = requests.get(url, headers=headers, allow_redirects=False)
    reponse.encoding = reponse.apparent_encoding
    return reponse.text

In [3]:
def find_target(target, string):
    """
    在string中获取所有符合正则表达式target的字符的集合（有重复字符）
    """

    re_compile = re.compile(target)
    result = re_compile.findall(string)
    return result

In [4]:
def find_name(name, string):
    """
    在string中获取第一个符合正则表达式name的字符串
    """
    return re.search(name, string)

In [5]:
def find_Chinese_or_digits(word):
    """
    对爬取得到的字符进行处理，如： '>3号航站楼站</'  ->  '3号航站楼站'
    """
    return ''.join(ch for ch in word if (('\u4e00' <= ch <= '\u9fff') or ch.isdigit()))

In [6]:
def find_table(text):
    """
    获取网页中车站信息列表
    """
    start = -1
    if text.find('车站列表</caption>') > 0:
        start = text.find('车站列表</caption>')
    elif text.find('车站列表</h3>') > 0:
        start = text.find('车站列表</h3>')
    elif text.find('车站列表<br/>') > 0:
        start = text.find('车站列表<br/>')
    elif text.find('车站信息</caption>') > 0:
        start = text.find('车站信息</caption>')
    elif text.find('车站信息</h3>') > 0:
        start = text.find('车站信息</h3>')

    end = start + text[start:].find('</table>')
    table = text[start:end]

    if '车站名称' in table:
        return table
    else:
        return find_table(text[start + 1:])  ##个别网页，《车站列表》下会有几个表格，前几个表格可能是无关信息，需舍去。

In [7]:
def get_line(url):
    """
    获取某条线路网页中的车站信息
    """
    TEXT = get_reponse_TEXT(url)

    linename = find_name('<title>北京地铁\w*线', TEXT).group()

    table = find_table(TEXT)

    line_pri = find_target('>\d*[\u4e00-\u9fa5]+站</', table)

    line = [find_Chinese_or_digits(entry) for entry in line_pri]
    linename = find_Chinese_or_digits(linename)
    return line, linename

In [8]:
def get_line_station_dict(url_source, target_source):
    reponse_text = get_reponse_TEXT(url_source)

    set_url_line = set(find_target(target_source, reponse_text))
    list_url_line = ['https://baike.baidu.com' + entry for entry in set_url_line]
    list_url_line.remove(
        'https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E8%A5%BF%E9%83%8A%E7%BA%BF'
    )  ##北京西郊线，网页无法爬取，故不作计算

    dict_line = {}
    i = 0

    for line_url in list_url_line:
        print(line_url)
        temp_line, temp_linename = get_line(line_url)
        i += 1

        if len(temp_line) > 0:
            dict_line[temp_linename] = temp_line
            print('linename= {}  ,success! i= {}'.format(temp_linename, i))
        else:
            print('linename= {}  ,defeated! i= {}'.format(temp_linename, i))

    return dict_line

In [9]:
##源网站url
url_source = 'https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485'
##各地铁线网页url正则表达式匹配符
target_source = '/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81\w*%E\d+.?\w*.?\w*.?E?\d*.?\w*.?\w*.?\w*%BA%BF'

In [10]:
dict_line = {}
dict_line = get_line_station_dict(url_source, target_source)

https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E4%BA%A6%E5%BA%84%E7%BA%BF
linename= 北京地铁亦庄线  ,success! i= 1
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E5%85%AB%E9%80%9A%E7%BA%BF
linename= 北京地铁八通线  ,success! i= 2
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%814%E5%8F%B7%E7%BA%BF
linename= 北京地铁4号线  ,success! i= 3
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8110%E5%8F%B7%E7%BA%BF
linename= 北京地铁10号线  ,success! i= 4
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E6%9C%BA%E5%9C%BA%E7%BA%BF
linename= 北京地铁机场线  ,success! i= 5
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%818%E5%8F%B7%E7%BA%BF
linename= 北京地铁8号线  ,success! i= 6
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81S1%E7%BA%BF
linename= 北京地铁1线  ,success! i= 7
https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8113%E5%8F%B7%E7%BA%BF
linename= 北京地铁13号线  ,success! i= 8
https://baike.baidu.com/item/

In [11]:
for name, list1 in dict_line.items():
    print(name)
    print(list1)
    print('\n')

北京地铁亦庄线
['宋家庄站', '肖村站', '小红门站', '旧宫站', '亦庄桥站', '亦庄文化园站', '万源街站', '荣京东街站', '荣昌东街站', '同济南路站', '宋家庄站', '亦庄火车站', '经海路站', '次渠南站', '次渠站', '亦庄火车站']


北京地铁八通线
['四惠站', '四惠东站', '高碑店站', '传媒大学站', '双桥站', '管庄站', '八里桥站', '通州北苑站', '果园站', '九棵树站', '梨园站', '临河里站', '土桥站', '施园站', '环球影城站']


北京地铁4号线
['天宫院站', '生物医药基地站', '义和庄站', '黄村火车站', '黄村西大街站', '清源路站', '枣园站', '高米店南站', '高米店北站', '西红门站', '新宫站', '公益西桥站', '角门西站', '马家堡站', '北京南站', '陶然亭站', '菜市口站', '宣武门站', '西单站', '灵境胡同站', '西四站', '平安里站', '新街口站', '西直门站', '动物园站', '国家图书馆站', '魏公村站', '人民大学站', '海淀黄庄站', '中关村站', '北京大学东门站', '圆明园站', '西苑站', '北宫门站', '安河桥北站']


北京地铁10号线
['巴沟站', '苏州街站', '海淀黄庄站', '知春里站', '知春路站', '西土城站', '牡丹园站', '健德门站', '北土城站', '安贞门站', '惠新西街南口站', '芍药居站', '太阳宫站', '三元桥站', '亮马桥站', '农业展览馆站', '团结湖站', '呼家楼站', '金台夕照站', '国贸站', '双井站', '劲松站', '潘家园站', '十里河站', '分钟寺站', '成寿寺站', '宋家庄站', '石榴庄站', '大红门站', '角门东站', '角门西站', '草桥站', '纪家庙站', '首经贸站', '丰台站', '泥洼站', '西局站', '六里桥站', '莲花桥站', '公主坟站', '西钓鱼台站', '慈寿寺站', '车道沟站', '长春桥站', '火器营站', '巴沟站']


北京地铁机场线
['东直门站', '三元桥站', '3号航站楼站', '2号航站楼站']




In [12]:
len(dict_line)

21

## 2.Search the optimal route 

### 不考虑每一站距离大小，仅根据 过站数 及 换线次数 ，寻找最优路径（我真爬不动了）

In [138]:
import collections

In [139]:
def is_goal(destation):
    """
    判断是否达到目标
    """
    
    def is_goal_(state):
        station, _, _, _ = state
        if station == destation:
            return True
        return False
    
    return is_goal_

In [140]:
def double(list_example):
    return [list_example[i:i+2] for i in range(len(list_example)-1)]

def subways(lines):
        dict_successors = collections.defaultdict(dict)
        for linename, stations in lines.items():
            for a, b in double(stations):
                dict_successors[a][b] = linename
                dict_successors[b][a] = linename
        return dict_successors

def successors(state):
    """
    给出此状态下所有可能动作，及动作结果
    """
    station, linename, changelinetimes, changestoptimes = state
    result = []
    for stop, lname in dict_successors[station].items():
        action = ' -> '.join([linename, lname])
        if lname != linename:
            result.append((action, (stop, lname, changelinetimes+1, changestoptimes+1)))
        else:
            result.append((action, (stop, linename, changelinetimes, changestoptimes+1)))
    return result

In [141]:
def comparison(path):
    """
    给出路径比较规则
    """
    _, _, changelinetimes, changestoptimes = path[-1]

    return (2*changelinetimes + changestoptimes)          ##换一次线路大致等于走过两站

In [142]:
def Search(state_start, is_goal, successors, comparison):
    paths = [[state_start]]
    seen = []
    while paths:
        path = paths.pop(0)
        state_now = path[-1]
        if is_goal(state_now):
            return path
        seen.append(state_now[0])
        for action, state in successors(state_now):
            if state[0] in seen:
                continue
            path_new = path + [action] + [state]
            paths.append(path_new)
        paths = sorted(paths, key=comparison, reverse = False)
    return False
    

In [143]:
dict_successors = subways(dict_line)

In [144]:
dict_successors

defaultdict(dict,
            {'宋家庄站': {'肖村站': '北京地铁亦庄线',
              '同济南路站': '北京地铁亦庄线',
              '亦庄火车站': '北京地铁亦庄线',
              '成寿寺站': '北京地铁10号线',
              '石榴庄站': '北京地铁10号线',
              '刘家窑站': '北京地铁5号线'},
             '肖村站': {'宋家庄站': '北京地铁亦庄线', '小红门站': '北京地铁亦庄线'},
             '小红门站': {'肖村站': '北京地铁亦庄线', '旧宫站': '北京地铁亦庄线'},
             '旧宫站': {'小红门站': '北京地铁亦庄线', '亦庄桥站': '北京地铁亦庄线'},
             '亦庄桥站': {'旧宫站': '北京地铁亦庄线', '亦庄文化园站': '北京地铁亦庄线'},
             '亦庄文化园站': {'亦庄桥站': '北京地铁亦庄线', '万源街站': '北京地铁亦庄线'},
             '万源街站': {'亦庄文化园站': '北京地铁亦庄线', '荣京东街站': '北京地铁亦庄线'},
             '荣京东街站': {'万源街站': '北京地铁亦庄线', '荣昌东街站': '北京地铁亦庄线'},
             '荣昌东街站': {'荣京东街站': '北京地铁亦庄线', '同济南路站': '北京地铁亦庄线'},
             '同济南路站': {'荣昌东街站': '北京地铁亦庄线', '宋家庄站': '北京地铁亦庄线'},
             '亦庄火车站': {'宋家庄站': '北京地铁亦庄线', '经海路站': '北京地铁亦庄线', '次渠站': '北京地铁亦庄线'},
             '经海路站': {'亦庄火车站': '北京地铁亦庄线', '次渠南站': '北京地铁亦庄线'},
             '次渠南站': {'经海路站': '北京地铁亦庄线', '次渠站': '北京地铁亦庄线'},
            

In [145]:
def begin_search(state_start, destation, is_goal, successors, comparison):
    return Search(state_start, is_goal(destation), successors, comparison)

In [146]:
state_start = ('国贸站', '北京地铁1号线', 0, 0)
begin_search(state_start, '国家图书馆站', is_goal, successors, comparison)

[('国贸站', '北京地铁1号线', 0, 0),
 '北京地铁1号线 -> 北京地铁1号线',
 ('永安里站', '北京地铁1号线', 0, 1),
 '北京地铁1号线 -> 北京地铁1号线',
 ('建国门站', '北京地铁1号线', 0, 2),
 '北京地铁1号线 -> 北京地铁2号线',
 ('朝阳门站', '北京地铁2号线', 1, 3),
 '北京地铁2号线 -> 北京地铁2号线',
 ('东四十条站', '北京地铁2号线', 1, 4),
 '北京地铁2号线 -> 北京地铁2号线',
 ('东直门站', '北京地铁2号线', 1, 5),
 '北京地铁2号线 -> 北京地铁2号线',
 ('雍和宫站', '北京地铁2号线', 1, 6),
 '北京地铁2号线 -> 北京地铁2号线',
 ('安定门站', '北京地铁2号线', 1, 7),
 '北京地铁2号线 -> 北京地铁2号线',
 ('鼓楼大街站', '北京地铁2号线', 1, 8),
 '北京地铁2号线 -> 北京地铁2号线',
 ('积水潭站', '北京地铁2号线', 1, 9),
 '北京地铁2号线 -> 北京地铁2号线',
 ('西直门站', '北京地铁2号线', 1, 10),
 '北京地铁2号线 -> 北京地铁大兴线',
 ('动物园站', '北京地铁大兴线', 2, 11),
 '北京地铁大兴线 -> 北京地铁大兴线',
 ('国家图书馆站', '北京地铁大兴线', 2, 12)]